# Поиск вырубок: разведка

Цель всех действий, которые производятся ниже -- понять, как выглядят вырубки на снимках, и какими призаками можно пользоваться, чтобы эти вырубки "поймать".

Пока я вижу следующие потенциально рабочие подходы:
 
1. Анализ временной динамики -- вырубка это резкое изменение отражательной способности. Причем изменение не любое, а определенного вида. Такой подход подразумевает, что мы можем обрабатывать каждый пиксель по одтельности, строить соответсвующие пикселю временные ряды изменений и искать на них "типичные" для вырубок изменения. Подход, по видимости, требует качественной фильтрации данных от мусора (облаков, теней) и уравнивания (стандартизации) снимков, сделанных в разное время (разное время года, разная освещенность, разные атмосферные условия). Представляется, что подобная предобработка данных будет довольно трудоемкой. *Текущий подход, который использует метрики, по сути является вариацией этого метода: метрики как раз и дают относительно устойчивые характеристики территории, которые сглаживают различия разновременных снимков.*
2. Анализ данных в пространственной области. Тут имеется в виду, что на снимках высокого разрешнения вырубки могут иметь характерный вид: центральная часть и идущие к ней волоки. Если это действительно так, то можно попробовать определять вырубки по подобным "отпечаткам". Это позволит избежать сложностей со стандартизацией разновременных снимков. С другой стороны, такой подход, вероятно, даст большое число ложных срабатываний на участках, которые похожи на вырубки (пролысины?, перекрестки дорог?). Тем не менее, мне представляется, что этот подход проще первого в реализации и, хотя первый путь потенциально более перспективен, я думаю, что нужно начать с поиска в пространственной области (легче реализовать => быстрее получим результаты => в случае удачи сэкономим время и усилия, а в случае неудачи потратимся не очень значительно).

Также возможны гибридные варианты, когда вырубка ищется и в пространственной, и во временной области.

Поэтому первая часть блокнота содержит раздел по визуализации имеющихся вырубок и визуального выделения признаков вырубок.

# Комментарий ИГ
1. Пока задача стоит разобратся с одним временем года (декабрь-март) и это с одной стороны проще - поскольку, нет фенологических изменений (за редким исключением), с другой стороны это сужает (возможно, на самом деле может быть и нет) возможности использования всяких индексов (NDVI и пр.). По первому подходу я вижу следующую последовательность действий (если это можно реализовать на данной платформе):

А) Создание базового безоблочного композита для зимы 2015 г. (если не получится набрать достаточное кол-во данных, добавить 2014). Здесь нам важно получить максимальное количество безоблачных пикселей и полное покрытие территории. Время отдельной сцены не важно.
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - удаление облаков и теней на первом этапе можно использовать BQA слой, но думаю, что качество не будет нас устраивать, можно попробовать добавить буфера (элементы best pixels selection -http://www.tandfonline.com/doi/full/10.1080/07038992.2014.945827). 

Другие варианты - добавить FMASK (если это можно сделать на этой платформе) или сделать отдельный модуль для классификации на облака/воду/тени и т.д. Я могу сделать нужный слой треннингов для терриитории.

Про нормализацию: сначала, можно попробовать сделать вообще без нее (главное без облаков) и посмотреть что получится, но если возникнет необходимость добавлять данные (L7,S2), то понадобится. 

Если следовать "правильному" пути, то нужно сделать два модуля:

1) пересчитывает значения в каналах в зависимости от положения от центра снимка (можно просто взять коэф. регрессии из Петиной статьи, например, поскольку этот параметр вряд ли должен менятся в зав. от сезона) 
2) Нормализация по МОДИС (MOD43 или MOD09) -отбор "лучших" пикселей и пересчет (в среднее или медианное) значение за этот же период (2015) - построение ур. регрессии (MOD ~ LANDSAT) -коэф. использовать при обработке.

Создание композита.

Б) Обработка актуальных данных
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - здесь пока можно ограничится только удалением облаков, поскольку изменения (рубки) в большинстве случаев сильно перекрывают всю прочую вариацию

Классификация

2. По второму подходу комментариев у меня пока нет

Планируется использовать -какие то текстурные характеристики брать с VHR и применять к Landsat -у?


## Подключение необходимых функций

Импортируем необходимые библиотеки и функции:

In [1]:
import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
    get_wms_link,
)

In [2]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

Для того, чтобы не засорять PERMANENT вспомогательными и/или временными данными, переключимся в отдельный MAPSET (под названием exploration). Если такого набора нет, то он будет создан (флаг -c):

In [3]:
grs.grass.run_command('g.mapset', mapset='exploration', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

exploration



## Как выглядят вырубки?

Для удобства отображения снимков воспользуемся WMS-сервисом. На снимки можно ссылаться следующим образом:

```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=MinX,MinY,MaxX,MaxY&WIDTH=...&HEIGHT=...&LAYERS=...">
```

Например:
```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">
```


Снимок1 | Снимок2
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B4@landsat">|<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">

Генерировать сылки можно при помощи вспомогательной функции:

In [18]:
print get_wms_link('toar_LC81120282015237LGN00_B5@landsat', 516684,5098286,543339.0,5113090.0)

http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=300&HEIGHT=300&LAYERS=toar_LC81120282015237LGN00_B5@landsat


### Рубки в PlanetLabs

#### Вырубки, появившиеся зимой 2015-2016.

(В таблицах ниже ссылки в заголовках ведут на интерактивную карту для более детального рассмотрения.)

* Вырубка, появившаяся зимой в промежутке между 8-го января и 3-го февраля 2016-го года. Вырубка едва заметна. Найти ее можно по более светлому пятну по сравнению с окружающим его участком. Сам участок помимо того, что поменял цвет, стал более "шершавый" на вид. Размер вырубки порядка 100 метров в диаметре. Элементы текстуры представляют собой округлые пятна, яркость которых увеличивается к середине пятна и спадает к его краям. Размер элемента текстуры обычно около 20 метров.

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.2648493647575&lat=47.61579915637495&zoom=17&styles=16,29) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.2648493647575&lat=47.61579915637495&zoom=17&styles=16,23)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160108_011229_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160302_225400_0c68_visual.composite@planet">



* Вырубка конца декабря 2015 года, размер приблизительно 50 метров по диагонали. Текстурные различия на снимках найти сложно, точнее на снимке от 7-го декабря также видна некоторая неоднородность структуры на фоне серой подстилки, но от фона она мало отличается. (Возможно, ошибка в данных и вырубка случилась ранее декабря?). Вот еще ссылки на эту рубку, приведенные здесь для того, чтобы можно было оценить вариации условий съемки: [29.02.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24662911891937&lat=47.558087982676&zoom=18&styles=16,29), [03.03.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24662911891937&lat=47.558087982676&zoom=18&styles=16,23).

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24662911891937&lat=47.558087982676&zoom=18&styles=16,41) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24662911891937&lat=47.558087982676&zoom=18&styles=16,35)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518184,5266924,518871,5267460&WIDTH=600&HEIGHT=600&LAYERS=20151207_013046_090c_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518184,5266924,518871,5267460&WIDTH=600&HEIGHT=600&LAYERS=20151230_042202_0c07_visual.composite@planet">



* Вырубка конца декабря 2016 г. Ниже приводится снимок, сделанный до появления вырубки и два снимка, сделанные позднее. Каких-либо заметных отличий вырубки от окружающего фона не заметно (на месте вырубки присутствует "шершавое" пятно, но оно также присутствует и на снимке, сделанном до вырубки). Ошибка в описании данных? Размер текстурного элемента также около 20 метров.

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24963855743408&lat=47.5705180572663&zoom=17&styles=16,29) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24963855743408&lat=47.5705180572663&zoom=17&styles=16,25) |  [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.24963855743408&lat=47.5705180572663&zoom=17&styles=16,23)
--------|---------|----
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160108_011229_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160229_040923_0c72_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160302_225400_0c68_visual.composite@planet">

* Вырубка, помеченная как произошедшая в конце декабря. Но, по всей видимости, она случилась в январе ([ссылка](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.27785003185272&lat=47.58894055913254&zoom=17&styles=16,29) на январский снимок, на котором присутствует объект, похожий на вырубку). Текстурные элементы также выражены слабо, их размер составляет около 20 метров.

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.27785003185272&lat=47.58894055913254&zoom=17&styles=16,37) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.27785003185272&lat=47.58894055913254&zoom=17&styles=16,23)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=520568,5270366,521223,5270901&WIDTH=600&HEIGHT=600&LAYERS=20151219_050318_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=520568,5270366,521223,5270901&WIDTH=600&HEIGHT=600&LAYERS=20160229_040923_0c72_visual.composite@planet">

#### Вырубки, появившиеся до конца осени 2015 г.

Ниже приводятся примеры вырубок, для которых нет данных PlanetLabs "до вырубки -- после вырубки", поэтому для них нельзя посмотреть направление изменений. Однако для этих примеров можно посмотреть, как выглядит вырубка на зимних снимках (почти) годом позднее. Поскольку вегетации в зимний период нет, то (ожидается), что на снимках представлено одно и то же состояние территории. Все различия, видимые на снимках, -- следствия различных условий съемки и состояния атмосферы. Поэтому эти примеры позволят оценить "внешнюю" вариативность данных, не связанную с реальными изменениями на местности.

* Вырубка, появившаяся в марте 2015 года. Ниже идут три зимних снимка одного и того же участка. Вырубка представлет собой относительно однородное пятно белого цвета.

[30.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.36266309924733&lat=47.68150010806235&zoom=17&styles=16,31) | [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.36266309924733&lat=47.68150010806235&zoom=17&styles=16,37) | [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.36266309924733&lat=47.68150010806235&zoom=17&styles=16,43)
--------|---------|----
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151230_043745_0c47_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151219_050318_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151111_235840_0c64_visual.composite@planet">

* Аналогичный случай, участок на километр юго-восточнее: [30.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.3673768043518&lat=47.675581447251545&zoom=17&styles=16,31), [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.3673768043518&lat=47.675581447251545&zoom=17&styles=16,37), [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.3673768043518&lat=47.675581447251545&zoom=17&styles=16,43). Вырубка -- однородное пятно белого цвета.

* Снимки одной и той же вырубки, сделанные в разные дни. Почти на всех снимках вырубки видна дорога (волок?), проходящая в центральной части вырубки, а также лесная дорога:
 (А) Самый поздний снимок из имеющихся [02.03.2016-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,21), вырубка на нем выглядит как белое пятно на темном фоне. По текстуре она очень напоминает облако. Снимок сделанный в тот же день, но другой path/row [02.03.2016-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,23), каких-то видимых различий между ними нет, не смотря на то, что вырубка на снимке "а" находится на самом краю сцены. (Б) Снимок имеет более четко выраженную текстуру фона идующу в северо-северо-восточном направлении: [29.02.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,25), по всей вероятности тот факт, что текстура видна не на всех снимках, каким-то образом связана с выпавшим (выпадающим?) снегом. (В) Очень похожая текстура присутствует на снимке [08.01.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,29), но она расположена в северо-серверо-восточном направлении; эта разница в направлении текстур (ср. снимок Б) говорит об их временной природе (осадки в момент их перемещения или освещение). Мне представляется маловероятным, что языки снега, подобные видимым на снимках, связаны с его переносом (выпадение осадков или поземка), поскольку выпадение снега должно быть связано с облачностью, а поземка маловероятна(?) в лесу. (Г) Далее идут три снимка, сделанные в один день но разными path/row: [30.12.2016-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,31), [30.12.2016-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,33), [30.12.2016-3](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,35), на всех трех присутствует та же вытяннутая в одном и том же направлении текстура. Судя по теням от горных цепей снимки сделаны приблизительно в одно и то же время, поэтому они не позволяют проверить связано ли направление текстур с освещенностью.  (Д) [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,37), (Е) Снимки [07.12.2015-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,39), [07.12.2015-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,41) мало что добавляют уже известному, но они сделаны в период, когда снега еще мало, поэтому на них вырубка видна более четко. (Ж) Снимок [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,43) является довольно мутным, поэтому несет не очень много информации. (З) Снимки [06.11.2015-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,45), [06.11.2015-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.310025872859&lat=47.58763618807765&zoom=17&styles=16,47) интересны тем, что они сделаны до того, как на территории устоялся снежный покров. Эти снимки показывают, что текстура вырубки практически не видна на фоне леса, также лесные дороги, четко прослеживаемые на зимних снимках на основной части протяжения скрыты пологом леса. В частности, на этих снимках не видна дорога (волок?), пересекающая вырубку, заметную на зимних снимках.

### Общий вывод по PlanetLabs

Вырубки на зимних снимках выглядят очень похожими на облака: имеют ту же "шершавую" структуру и белый цвет. Они могут отличаться по текстуре от фона, но не всегда. Зачастую вырубки теряются на фоне других белых объектов, но часть из таких фоновых объектов можно отсеять по их линейной структуре: они занимают большие площади и их текстуры имеют характерные вытянутости вдоль одной из осей. Текстура вырубки (если она присутствует) представляет собой округлые пятна, яркость которых увеличивается к середине пятна и спадает к его краям. Размер элемента текстуры обычно около 20 метров, хотя он может варьироваться даже в пределах одной вырубки (от 15 и до 30, реже 40 метров).

## Применение текстурных признаков для поиска вырубок

Оценим применимость текстурных признаков для поиска вырубок. В качестве первого приближения возьмем классические текстурные характеристики, введенные в работе Haralick, R.M., K. Shanmugam, and I. Dinstein (1973). Textural features for image classification. IEEE Transactions on Systems, Man, and Cybernetics, SMC-3(6):610-621.

### Описание текстурных признаков

#### Схема рассчета текстурных признаков

Эти текстурные признаки рассчитываются для одноканального растра по следующей схеме:
 * Задаются параметры алгоритма: размер скользящего окна, и вектор сдвига для расчета статистики. Вектор сдвига определяется относительно текущей ячейки (пикселя) скользящего окна и представляет собой число ячеек и угол на которые нужно сместиться от текущей ячейки (например, сдвиг на 90 градусов и три пикесля "помечает" ячейку, находящуюся тремя пикселями севернее относительно текущего пикселя).
 * Далее строится гистограмма совместного распределения яркости пар пикселей $P$, находящихся на заданном расстоянии и направлении друг от друга. Эта гистограмма представляет собой таблицу, число строк и столбцов таблицы равно числу градаций серого в анализируемом растре. Изначально таблица инициализируется нулями. Затем в текущем окне пробегаются пары пикселей, находящиеся друг от друга на расстоянни и направлении с указанным сдвигом, и заполняется таблица: если начальный пиксель имеет значение яркости I, а парный ему (соответствующий сдвигу) пиксель -- яркость J, то значение в таблице на I-й строке в J-м столбце увеличивается на единицу. Таким образом после завершения процедуры в каждой ячейки таблицы будет хранится число пар пикселей, разделенных сдвигом, с соответсвующими яркостями (гистограмма совместного распределения). Математически процедура записывается следущим образом (на пример направления в 0 градусов): $$P(i,j,d,0) = \#\Big\{\big((k,l), (m,n)\big) \in (L_y\times L_x)\times (L_y\times L_x): k-m=0, |l-n|=d, I(k,l)=i, I(m,n)=j \Big\},$$ где знаком $\#$ обозначено число пикселей, функция $I(k,l)$ возвращает яркость пикселя с координатами $(k,l)$, а $(L_y\times L_x)$ -- окно, в котором происходит анализ.
 * По полученной гистограмме рассчитываются текстурные признаки, представленные ниже.

#### Формулы расчета и качественное описание признаков

Текстурные признаки по Харалику представляют собой статистики, построенные по гистограмме совместного распределения яркостей пикселей. Эти признаки сильно коррелированы между собой. Для большей наглядности и понимания на качественном уровне того, как ведут себя признаки, удобно представлять себе два крайних случая -- почти полностью однородное изображение (все пиксели примерно одинаковой яркости) и полностью случайное, когда яркость всех пикселей распределена по равномерному закону, не зависит от положения пикселя.

Для первого случая матрица $P$ будет представлять собой матрицу, в которой один элемент содержит очень большое значение, а остальные элементы близки к нулю. Для второго случая матрица $P$ будет состоять из примерно равных значений.

##### Обозначения

 * $N_g$ -- число градаций серого в растре.
 * $P$ -- гистограмма совместного распределения яркости пар пикселей, находящихся на заданном расстоянии и направлении друг от друга.
 * $R$ -- общее число пар пикселей, находящихся на заданном расстоянии и направлении друг от друга (зависит от размера окна, расстояния и направления сдвига).
 * $p(i,j)$ -- ячейка $(i,j)$ нормированной гистограммы $P$, (т.е. $p(i,j) = P(i,j)/R$).
 * $p_x(i)$ -- сумма значений в i-й строке матрицы $p(i,j)$, т.е. $p_x(i) = \sum_{j=1}^{N_g}p(i,j)$.
 * $p_y(j) = \sum_{i=1}^{N_g}p(i,j)$.
 * $p_{x+y}(k) = \sum_{i=1}^{N_g}\sum_{j=1}^{N_g}p(i,j) \quad \Big| i+j = k$ (здесь суммирование идет только по тем парам i, j, которые в сумме дают k: $i+j=k$, $k=2,3,\dots, 2N_g$).
 * $p_{x-y}(k) = \sum_{i=1}^{N_g}\sum_{j=1}^{N_g}p(i,j) \quad \Big| |i-j| = k$, $k=0,1,\dots, N_g-1$. Аналогично, суммирование идет только по тем парам, модуль разности которых равен $k$.
 
 
 Для сокращения записи ниже также применяются следующие обозначения: $\sum_i = \sum_{i=1}^{N_g}$,  $\sum_j = \sum_{j=1}^{N_g}$

##### Признаки

*Заметим, что $R$ будет расти как квадрат от размера стороны скользящего окна. $N_g$ -- число градаций серого в растре, обычно оно не велико (255 уже много).*

Angular Second Moment:
$$f_1 = \sum_i \sum_j \big(p(i,j)\big)^2.$$

Для случая однородной текстуры значение $f_1$ будет приближаться к единице, для случайной текстуры -- к $1/R$, при больших размерах окна -- к нулю.

Contrast:
$$f_2 = \sum_{n=0}^{N_g-1}n^2 p_{x-y}(n).$$

Для случая однородной текстуры значение $f_2$ будет приближаться к нулю, для случайной текстуры -- увеличиваться с ростом размера окна *(с последним не уверен на 100%, нужно еще посчитать)*.

Correlation:
$$f_3 = \frac{\sum_{i}\sum_{j} (ij)p(i,j) - \mu_x \mu_y}{\sigma_x\sigma_y},$$
где $\mu_x$ и $\mu_y$ -- среднее для $p_x$ и $p_y$ соотвественно, а $\sigma_x$ и $\sigma_y$ -- их стандартные отклонения.

Для однородной текстуры $f_3$ будет очень велико (для идеально однородной -- получится деление на ноль), для случайной текстуры будет около нуля.

Sum of Squares:
$$f_4 = \sum_{i}\sum_{j} (i-\mu)^2 p(i,j).$$

Inverse Difference Moment:
$$f_5 = \sum_{i}\sum_{j} \frac{1}{1+(i-j)^2} p(i,j).$$

Sum Average:
$$f_6 = \sum_{i=2}^{2N_g} i p_{x+y}(i).$$

Sum Variance:
$$f_7 = \sum_{i=2}^{2N_g} (i-f_8)^2 p_{x+y}(i).$$

Sum Entropy:
$$f_8 = - \sum_{i=2}^{2N_g} p_{x+y}(i)\log \big(p_{x_y}(i)\big).$$

Entropy:
$$f_9 = -\sum_{i}\sum_{j} p(i,j) \log \big( p(i,j)\big).$$

Difference Variance:
$$f_{10} = variance(p_{x-y}).$$

Difference Entropy:
$$f_{11} = - \sum_{i=0}^{N_g -1} p_{x-y}\log p_{x-y}(i).$$

Information Measure of Correlation 1:
$$f_{12} = \frac{HXY - HXY1}{\max(HX, HY)}.$$
Information Measure of Correlation 2:
$$f_{13} = \Big( 1- e^{-2(HXY2 - HXY)} \Big)^{1/2}.$$
Где
$HXY = - \sum_{i}\sum_{j} p(i, j) \log p(i,j)$, 
$HXY1 = - \sum_{i}\sum_{j} p(i, j) \log \big(p_x(i)p_y(j)\big)$ и 
$HXY2 = - \sum_{i}\sum_{j} p_x(i)p_y(j)\log\big(p_x(i)p_y(j)\big)$.

Maximal Correlation Coefficient:
$f_{14}$ -- корень из второго наибольшее собственное значение $Q$, где
$$Q(i,j) = \sum_k \frac{p(i,k)p(j,k)}{p_x(i) p_y(k)}.$$

### Пример построения текстурных признаков

В качестве примера возьмем [следующий участок территории с вырубкой](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&lon=135.2652&lat=47.6158&angle=0&zoom=16&styles=16,23). Поскольку расчет текстурных признаков достаточно трудоемкая процедура, то ниже будут приводиться только команды GRASS без их непосредственного исполнения на сервере. Результаты будут показаны в виде встроенных изображений.

#### Выбор канала

Текстурные признаки строятся по одноканальному растру. Данные PlanetLabs представляют собой RGB изображение в естесственных цветах. Поэтому мы можем использовать для вычислений какой-либо из каналов или же создать смешанное одноканальное изображение на основе отдельных (например, использовать взвешенное среднее от трех каналов).

Поскольку синий канал наиболее подвержен влиянию атмосферных эффектов, было решено не использовать его в работе. Из оставшихся двух каналов для экспериментов был ипользован красный канал изображения. (*Почему? Не знаю -- нужно же было с чего-то начать. Возможно, есть объективные основания для выбора канала или построение смешанного изображения, но я не лез далеко в эту тему*)

#### Выбор числа градаций серого

Число градаций серого в растре -- очень важная характеристика при построении текстуры, посколько она определяет размер матрицы $P$, по которой строятся текстурные призаки. Поэтому если эта матрица содержит слишком мало элементов, то мы получим излишне грубое описание текстур. Однако использование большой матрицы тоже не оправдано -- для того, чтобы получить устойчивые признаки (т.е. признаки, которые не будут значительно меняться при незначительных вариациях текстур) требуется оценить величины элеменов матрицы с достаточной точностью. Точность этой оценки зависит в первую очередь от числа примеров $R$ (т.е. размера скользящего окна), на основе которых строится матрица $P$. Таким образом изменяя используемое число градаций серого в анализируемом растре мы влияем на конечную точность классификации на основе текстурных признаков (мы же строим признаки для последующей классификации). Ожидается, что увеличивая число градаций с небольших начальных значений, мы будем увеличивать точность классификации, но, после прохождения некоторого порога, она начнет падать. Поиск оптимального числа градаций -- отдельный вопрос, который нужно исследовать (им будет смысл заниматься, если сам подход себя оправдает). Ниже же число градаций подбирается "на глаз".

Заметим также, что число элементов матрицы $P$ растет как квадрат числа градаций серого.

Команды приводятся для возможности воспроизвести результат, выбор параметров команд (помимо градаций серого) описывается ниже.
```
r.mapcalc expression="reduced4 = int((20160302_225400_0c68_visual.red - 117) / 4)"
r.mapcalc expression="reduced8 = int((20160302_225400_0c68_visual.red - 117) / 8)"
r.mapcalc expression="reduced8 = int((20160302_225400_0c68_visual.red - 117) / 16)"

r.texture -a input="20160302_225400_0c68_visual.red" output="textures_init_6" size=27 distance=6
r.texture -a input="reduced4" output="textures_red4_6" size=27 distance=6
r.texture -a input="reduced8" output="textures_red8_6" size=27 distance=6
r.texture -a input="reduced16" output="textures_red16_6" size=27 distance=6
```

Результат расчета Angular Second Moment для трех растров представлен в таблице:

Основа для построения |Количество градаций| Результат 
---------|---------
Исходный растр | 256 |<img src='https://176.9.38.120/cruncher/files/deforestation/Img/Textures/ASM_d6_init.png'> 
Reduced4|34|<img src="https://176.9.38.120/cruncher/files/deforestation/Img/Textures/ASM_d6_red4.png"> 
Reduced8|17|<img src="https://176.9.38.120/cruncher/files/deforestation/Img/Textures/ASM_d6_red8.png"> 
Reduced16|8|<img src="https://176.9.38.120/cruncher/files/deforestation/Img/Textures/ASM_d6_red16.png"> 

Из таблицы видно, что 256 градаций слишком много, поскольку все детали изчезли. В то же время 8 градаций, по всей вероятности, слишком мало, поскольку разнородные объекты становятся слишком похожими. В качестве основы для дальнейших экспериментов было выбрано 17 градаций.